# Classification and Sequence Labelling (50 marks)

In this coursework you will apply some of the methods learned during the
unit, and explore their behaviour using some real-world datasets. For the
first set of tasks, we will be working with the dataset “Activity recognition
with healthy older people using a batteryless wearable sensor”. This dataset
contains sequences of readings taken from several sensors on a device worn
by a person as they move about in a room. The goal is to predict the activity
label for each data point, i.e., what the person was doing, which may be 1:
sit on bed, 2: sit on chair, 3: lying, or 4: ambulating. To help you accessm
the data, we provide a notebook “activity recognition.ipynb” in our Github
repository (the same repository as the lab worksheets). The notebook code
loads the data into a suitable format for the following tasks.